In [1]:
import numpy as np
import pandas as pd
import sys,os

def load_files(batch):
    images = []
    labels = []
    for i in batch:
        print('Loading File: ' + i)
        x = np.load(i,encoding = 'latin1',allow_pickle=True).item()
        keys = x.keys()
        for key in keys:
            images.append(x[key][0])
            labels.append(x[key][1])
    return np.array(images),np.array(labels)

def get_feature(labels,feature):
    feature_values = []
    for i in labels:
        feature_values.append(i[feature])
    feature_values = np.array(feature_values)
    return feature_values

def get_cos_values(zenith,azimuth):
    cos1 = []
    cos2 = []
    cos3 = []
    for i,j in zip(zenith,azimuth):
        cos1.append(np.sin(i) * np.cos(j))
        cos2.append(np.sin(i) * np.sin(j))
        cos3.append(np.cos(i))
    return np.array(cos1),np.array(cos2),np.array(cos3)

In [2]:
file_path = '/fs/scratch/PAS1495/amedina/'
y = os.listdir(file_path+'processed_3D')

file_names = []

for i in y:
    file_names.append(file_path+'processed_3D/'+i)

file_names_batched = list(np.array_split(file_names,100))

In [3]:
images,labels = load_files(file_names_batched[0])

images = images[:,:,:,:,4:7]

zenith_values = get_feature(labels,1)
azimuth_values = get_feature(labels,2)

cos1,cos2,cos3 = get_cos_values(zenith_values,azimuth_values)

Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5958.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_207.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5774.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5256.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4768.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_2519.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4632.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5356.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4155.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_5633.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_4163.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_7039.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_6819.npy
Loading File: /fs/scratch/PAS1495/amedina/processed_3D/images_3831.npy
Loading

In [4]:
images.shape

(3303, 60, 10, 19, 3)

In [5]:
import tensorflow as tf

import keras
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv3D, MaxPooling3D ,BatchNormalization, GlobalAveragePooling3D
from keras import backend as K
from sklearn.model_selection import train_test_split,KFold
from keras.layers import LeakyReLU, PReLU
from keras import regularizers

Using TensorFlow backend.


In [6]:
cos_values = np.array(list(zip(cos1,cos2,cos3)))

In [7]:
def get_data(images,values):
    x_train, x_test , y_train , y_test = train_test_split(images,values,test_size = 0.2 , random_state=42)
    return x_train,x_test,y_train,y_test

x_train,x_test,y_train,y_test = get_data(images,cos_values)

In [8]:
def loss_space_angle(y_true,y_pred):
    subtraction = tf.math.subtract(y_true,y_pred)
    y = tf.matrix_diag_part(K.dot(subtraction,K.transpose(subtraction)))
    loss = tf.math.reduce_mean(y)
    return loss


In [9]:
batch_size = 128
epochs=100

img_heights,img_rows, img_cols = 60,10,19
input_shape = (img_rows, img_cols)

kernel = 3
kernel2 = 2

model = Sequential()
model.add(Conv3D(16,kernel,
                 input_shape = (img_heights,img_rows,img_cols,3),
                 padding='same',
                 kernel_regularizer=regularizers.l2(0.01)))
                #activity_regularizer=regularizers.l1(0.01)))

model.add(LeakyReLU(alpha = 0.01))
model.add(MaxPooling3D(kernel2,padding='same'))
model.add(Conv3D(32,kernel,padding='same'))
model.add(MaxPooling3D(kernel2,padding='same'))
model.add(Conv3D(32,kernel,padding='same'))
model.add(MaxPooling3D(kernel2,padding='same'))
model.add(Conv3D(32,kernel,padding='same'))
model.add(MaxPooling3D(kernel2,padding='same'))
model.add(Conv3D(32,kernel,padding='same'))

model.add(Dropout(rate=1-0.5))

model.add(LeakyReLU(alpha = 0.01))
model.add(MaxPooling3D(kernel2,padding='same'))
model.add(Conv3D(32,kernel,padding='same'))
model.add(MaxPooling3D(kernel2,padding='same'))
model.add(Conv3D(32,kernel,padding='same'))
model.add(MaxPooling3D(kernel2,padding='same'))
model.add(Conv3D(32,kernel,padding='same'))
model.add(MaxPooling3D(kernel2,padding='same'))
model.add(Conv3D(32,kernel,padding='same'))

#model.add(GlobalAveragePooling3D())
model.add(Flatten())

model.add(Dense(16,activation='linear'))
model.add(LeakyReLU(alpha = 0.01))
model.add(Dense(3))

#opt = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=.09, nesterov=True)
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-6, amsgrad=False)
model.compile(optimizer=opt , loss = loss_space_angle)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
session_conf = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8)
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
keras.backend.set_session(sess)

history=model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_data=(x_test, y_test))

Train on 2642 samples, validate on 661 samples
Epoch 1/100
2560/2642 [============================>.] - ETA: 1s - loss: 0.8954

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
print(history_dict.keys())
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values)+1)

plt.plot(epochs,loss_values,'bo',label='Training loss')
plt.plot(epochs,val_loss_values,'b',label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()